In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/all_quali_2024.csv")

In [11]:
import pandas as pd

driver_team_map = [
    (1, "Max Verstappen", "RED BULL"),
    (11, "Sergio Perez", "RED BULL"),
    (16, "Charles Leclerc", "FERRARI"),
    (55, "Carlos Sainz", "FERRARI"),
    (63, "George Russell", "MERCEDES"),
    (44, "Lewis Hamilton", "MERCEDES"),
    (4, "Lando Norris", "MCLAREN"),
    (81, "Oscar Piastri", "MCLAREN"),
    (14, "Fernando Alonso", "ASTON MARTIN"),
    (18, "Lance Stroll", "ASTON MARTIN"),
    (23, "Alexander Albon", "WILLIAMS"),
    (2, "Logan Sargeant", "WILLIAMS"),
    (31, "Esteban Ocon", "ALPINE"),
    (10, "Pierre Gasly", "ALPINE"),
    (20, "Kevin Magnussen", "HAAS"),
    (27, "Nico Hulkenberg", "HAAS"),
    (24, "Zhou Guanyu", "SAUBER"),
    (77, "Valtteri Bottas", "SAUBER"),
    (3, "Daniel Ricciardo", "RB"),
    (22, "Yuki Tsunoda", "RB")
]

df_map = pd.DataFrame(driver_team_map, columns=["driver_number", "driver_name", "team"])
df_map.to_csv("../data/driver_team_mapping.csv", index=False)

print("✅ Static team mapping created")


✅ Static team mapping created


In [12]:
teams = pd.read_csv("../data/driver_team_mapping.csv")
df = df.merge(teams, on="driver_number", how="left")


In [13]:
# Identify unique meetings
meetings = sorted(df["meeting_key"].unique())
train_meetings = meetings[:4]   # first 4 races
test_meeting = meetings[4]      # 5th race

df_train = df[df["meeting_key"].isin(train_meetings)]
df_test  = df[df["meeting_key"] == test_meeting]


In [14]:
driver_features = df_train.groupby("driver_number").agg(
    driver_avg_lap = ("lap_duration", "mean"),
    driver_best_lap = ("lap_duration", "min"),
    driver_avg_s1 = ("duration_sector_1", "mean"),
    driver_avg_s2 = ("duration_sector_2", "mean"),
    driver_avg_s3 = ("duration_sector_3", "mean"),
    driver_avg_i1 = ("i1_speed", "mean"),
    driver_avg_i2 = ("i2_speed", "mean"),
    driver_avg_st = ("st_speed", "mean"),
    driver_lap_std = ("lap_duration", "std")
).reset_index()

driver_features["driver_consistency"] = 1 / driver_features["driver_lap_std"]


In [15]:
team_features = df_train.groupby("team").agg(
    team_avg_lap = ("lap_duration", "mean"),
    team_best_lap = ("lap_duration", "min"),
    team_avg_s1 = ("duration_sector_1", "mean"),
    team_avg_s2 = ("duration_sector_2", "mean"),
    team_avg_s3 = ("duration_sector_3", "mean"),
    team_avg_i1 = ("i1_speed", "mean"),
    team_avg_i2 = ("i2_speed", "mean"),
    team_avg_st = ("st_speed", "mean"),
).reset_index()


In [16]:
df_train = df_train.merge(driver_features, on="driver_number", how="left")
df_train = df_train.merge(team_features, on="team", how="left")

df_test = df_test.merge(driver_features, on="driver_number", how="left")
df_test = df_test.merge(team_features, on="team", how="left")


In [17]:
df_train.to_csv("../data/train_enriched.csv", index=False)
df_test.to_csv("../data/test_enriched.csv", index=False)
